In [1]:
import datajoint as dj
import numpy as np
import trimesh

m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

Connecting celiib@10.28.0.34:3306


In [2]:
segment_id = 107816118160698192
version = 0
lookup_key = dict(segment_id=segment_id, version = version)

# Function for Importing the Actual Annotations

In [3]:
# function that helps import the actual labels
import pandas as pd
def retrieve1(core_segment_id, version=-1):
        #print("core_segment_id = " + str(core_segment_id))
        #print("version = " + str(version))
        rel = m65.FromNeuromancerMergeError & dict(segment_id=core_segment_id)
        if version == -1:
            rel &= dj.U('segment_id').aggr(rel.proj(), version='max(version)')
        else:
            rel &= dict(version=version)

        if len(rel) == 1:
            csv_string = rel.fetch1('annotations').item()
            
            from io import StringIO
            
            with StringIO(csv_string) as csv_io:
                ano_df = pd.read_csv(csv_io)
            return ano_df
        else:
            return None

# functions for creating the points and the 3D line

In [12]:
# How to create the points and the lines: 
# box maker function
def create_synapse_box(center,side_length):
    """
    Purpose: Will create the mesh variables of vertices and faces
    for a box with a specified center and side length
    
    Parameters:
    center (tuple/np.array) : list of the x,y,z coordinates of the box
    side_length(float)      : the length that the sides need to be of box
    
    Returns: 
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for the box
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for the box
    
    How to use function: 
    create_synapse_box((2,2,2),2.5)
    """
    
    unit_box_vertices = np.array([
        [0,0,0],
        [1,0,0],
        [1,1,0],
        [0,1,0],
        [0,0,1],
        [1,0,1],
        [1,1,1],
        [0,1,1],
    ])

    unit_box_faces = np.array([
        [0,4,5],
        [0,5,1],
        [1,5,6],
        [1,6,2],
        [2,6,7],
        [2,7,3],
        [3,7,4],
        [3,4,0],
        [5,4,7],
        [6,5,7],
        [1,2,0],
        [3,0,2]

        ])

    #Now need to shift it down to center
    unit_box_vertices = unit_box_vertices - 0.5
    
    unit_box_vertices = unit_box_vertices*side_length
    unit_box_vertices = unit_box_vertices + np.array(center)
    
    return unit_box_vertices,unit_box_faces



In [22]:
import ast


# How to create the points and the lines: 
# box maker function

def calculate_normal_vector(p1,p2,p3):
    A = p2 - p1
    A = A/np.linalg.norm(A)
    B = p3 - p1
    B = B/np.linalg.norm(B)
    
    #print("A.shape = " + str(A.shape))
    #print("B.shape = " + str(B.shape))

    normal = np.cross(A,B)
    normal = normal/np.linalg.norm(normal)
    return normal

#now need to find the rotation needed
def rotation_vp3(a,b):
    a = a/np.linalg.norm(a)
    b = b/np.linalg.norm(b)
    

    
    if np.dot(a,b) == 1 or np.dot(a,b) == -1:
        return np.identity(len(a))
    
    #print("a.shape = " + str(a.shape))
    #print("b.shape = " + str(b.shape))
    
    v = np.cross(a,b)
    #print("v = " + str(v))
    s = np.linalg.norm(v)
    c = np.dot(a,b)
    
    v_matrix = np.array([
        [0, -v[2] , v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])
    
    v_matrix_squared = v_matrix @ v_matrix
    
    R = np.identity(len(a)) + v_matrix + v_matrix_squared * (1/(1+c))
    
    return R

def create_annotation_line(center_a,center_b,side_length):
    """
    Purpose: Will create the mesh variables of vertices and faces
    for a box with a specified center and side length
    
    Parameters:
    center (tuple/np.array) : list of the x,y,z coordinates of the box
    side_length(float)      : the length that the sides need to be of box
    
    Returns: 
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for the box
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for the box
    
    How to use function: 
    create_synapse_box((2,2,2),2.5)
    """
    center_a = np.array(center_a)
    center_b = np.array(center_b)
    

    face_vertices_a = np.array([
        [0,1,1],
        [0,1,-1],
        [0,-1,1],
        [0,-1,-1]
    ])
    
    
    #Now need to shift it down to center
    face_vertices_a = face_vertices_a*side_length
    
    #apply the necessary rotation (skipping for now)
    direction_of_line = center_b - center_a
    direction_of_line = direction_of_line/np.linalg.norm(direction_of_line)
    #print("direction_of_line = " + str(direction_of_line))
    #print("direction_of_line = " + str(direction_of_line.shape))
    
    # normal vector is just the cross product of 2 lines on the traingle (ALWAYS STAYS SAME NORMAL AT BEGINNING)
    p1 = np.array([0,1,1])
    p2 = np.array([0,1,-1])
    p3 = np.array([0,-1,1])
    
    normal = calculate_normal_vector(p1,p2,p3)

    #print("normal = " + str(normal.shape))
    #print("normal = " + str(normal))

    rotation_matrix_for_normal = rotation_vp3(normal,direction_of_line)
    
    #rotate the points and then calculate rotation matrix 
    face_vertices_a = (rotation_matrix_for_normal @ face_vertices_a.T).T
    #face_vertices_b = rotation_matrix_for_normal @ face_vertices_b.T
    face_vertices_b = face_vertices_a.copy()
    

    #verify that both the faces have the normals that are in same direction as beginning line
    rotated_normal_a = calculate_normal_vector(face_vertices_a[0].reshape(-1,),face_vertices_a[1].reshape(-1,),face_vertices_a[2].reshape(-1,))
    #print("np.dot(rotated_normal_a,direction_of_line) = " + str(np.dot(rotated_normal_a,direction_of_line)))
    #rotated_normal_b = calculate_normal_vector(face_vertices_b[0],face_vertices_b[1],face_vertices_b[2])
    #print("np.dot(rotated_normal_b,direction_of_line) = " + str(np.dot(rotated_normal_b,direction_of_line)))
    
    
    line_faces = np.array([
        [1,2,3],
        [2,3,4],
        [5,6,7],
        [6,7,8],
        [5,1,7],
        [1,3,7],
        [6,8,2],
        [2,8,4],
        [6,2,5],
        [2,1,5],
        [8,7,4],
        [4,7,3]
    ])
    
    line_faces = line_faces - 1 #makes all the vertices 0 indexed

    
    
    #add shift the face vertices to the correct center points
    shifted_face_vertices_a =  face_vertices_a + center_a
    shifted_face_vertices_b = face_vertices_b + center_b
    
    #stack into final vertices list 
    line_vertices = np.vstack([shifted_face_vertices_a,shifted_face_vertices_b])
    
    
    return line_vertices,line_faces
    

In [23]:
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    #print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

# Function that creates the annotation mesh

In [24]:
def get_annotation_mesh(segment_id,version,side_length):
    df = retrieve1(segment_id,version)
    
    line_data = df[df["type"] != "point"][["pointA","pointB"]].to_numpy()
    import ast
    converted_line_array = np.array([ast.literal_eval(p) for p in line_data.ravel()]).reshape(-1,2,3)
    
    """
    How to iterate through them: 
    for a,b in converted_line_array:
        #can call the create function from here
        print(a)
        print(b)
    """
    
    points_data = df[df["type"] == "point"][["point"]].to_numpy()
    
    converted_point_array = np.array([ast.literal_eval(p) for p in points_data.ravel()]).reshape(-1,3)
    
    """
    How to iterate through them:
    
    for a in converted_point_array:
        #can call the create function from here
        print(a)
    """
#     print(converted_point_array)
#     print(converted_line_array)
    
    #assembles all of the mesh into a list
    annotation_vertices = []
    annotation_faces = []
    
    for point in converted_point_array:
        unit_box_vertices,unit_box_faces = create_synapse_box(point,side_length)
        annotation_vertices.append(unit_box_vertices)
        annotation_faces.append(unit_box_faces)

    for a,b in converted_line_array:
        unit_box_vertices,unit_box_faces = create_annotation_line(a,b,side_length)
        annotation_vertices.append(unit_box_vertices)
        annotation_faces.append(unit_box_faces)
        
    
    #concatenates all the meshes into one mesh piece
    box_mesh_verts, box_mesh_faces = add_mesh_piece(annotation_vertices[0],annotation_faces[0],annotation_vertices[1:],annotation_faces[1:])
    
    return box_mesh_verts, box_mesh_faces

In [26]:
segment_id = 107816118160698192
version = 0
side_length = 1000

box_mesh_verts, box_mesh_faces = get_annotation_mesh(segment_id,version,side_length)

import trimesh
new_mesh = trimesh.Trimesh(vertices = box_mesh_verts, faces = box_mesh_faces)
new_mesh.export("annotations.off")
print("hello")

hello
